In [1]:
import pandas as pd

In [2]:
threshold = -90
n = 2   # number of seconds
cluster_members = 3 # number of elements together

In [14]:
df = pd.read_excel("rxlevel.xlsx",index_col=0)

In [4]:
df.reset_index(drop=True,inplace=True)

In [5]:
df2 = df.copy()

In [6]:
df2.head()

,Time,Date,Latitude,Longitude,Categorized Ec/Io:A1,Categorized RSCP:A1
0,02:51:31.000,7/5/2022,31.43781,73.139322,-4.7,-62.099998
1,02:51:33.000,7/5/2022,31.43781,73.139322,-6.5,-63.700001
2,02:51:35.000,7/5/2022,31.43781,73.139322,-8.0,-63.900002
3,02:51:37.000,7/5/2022,31.43781,73.139322,-6.9,-63.299999
4,02:51:39.000,7/5/2022,31.43781,73.139322,-8.2,-64.000000


In [7]:
#df2.rename(columns={"RxLevFull (dBm) - .Server":"frequency"},inplace=True)

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27114 entries, 0 to 27113
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Time                 27114 non-null  object 
 1   Date                 27114 non-null  object 
 2   Latitude             27114 non-null  float64
 3   Longitude            27114 non-null  float64
 4   frequency            27114 non-null  float64
 5   Categorized RSCP:A1  27053 non-null  float64
dtypes: float64(4), object(2)
memory usage: 1.2+ MB


In [9]:
green_df  = df2.loc[df2["RxLevFull (dBm) - .Server"] >= threshold]
green_df.shape

(23689, 6)

In [13]:
red_df = df2.loc[df2["RxLevFull (dBm) - .Server"] < threshold]
red_df.head()

,Time,Date,Latitude,Longitude,frequency,Categorized RSCP:A1
326,03:06:11.000,7/5/2022,31.424067,73.156906,-12.1,-69.400002
351,03:07:01.000,7/5/2022,31.427028,73.153051,-13.4,-78.599998
352,03:07:03.000,7/5/2022,31.427156,73.152885,-12.9,-75.199997
519,03:12:45.000,7/5/2022,31.436808,73.147370,-13.0,-76.800003
527,03:13:01.000,7/5/2022,31.437429,73.148028,-13.2,-73.699997


In [10]:
red_df= red_df.sort_values(by="Time")

In [11]:
red_df.head()

,Time,Date,Latitude,Longitude,frequency
28412,12:35:05.000,7/14/2022,31.379836,73.085556,-90.900002
28413,12:35:07.000,7/14/2022,31.379958,73.085677,-91.599998
28414,12:35:09.000,7/14/2022,31.380079,73.085810,-91.400002
28415,12:35:11.000,7/14/2022,31.380208,73.085924,-90.300003
24403,17:50:53.000,7/13/2022,31.411087,73.094298,-90.199997


In [12]:
red_df["era"] = pd.to_datetime(red_df["Date"]+ " " +red_df["Time"], infer_datetime_format=True,)


In [14]:
red_df = red_df.sort_values(by="era")
red_df["Group"] = red_df.era.diff().dt.seconds.gt(n).cumsum()
red_df
#red_df.dtypes
#pd.to_datetime(red_df["Time"], format = " %d/%m/%Y %H:%M:%S ",errors="coerce"  )

,Time,Date,Latitude,Longitude,frequency,era,Group
24403,17:50:53.000,7/13/2022,31.411087,73.094298,-90.199997,2022-07-13 17:50:53,0
28412,12:35:05.000,7/14/2022,31.379836,73.085556,-90.900002,2022-07-14 12:35:05,1
28413,12:35:07.000,7/14/2022,31.379958,73.085677,-91.599998,2022-07-14 12:35:07,1
28414,12:35:09.000,7/14/2022,31.380079,73.085810,-91.400002,2022-07-14 12:35:09,1
28415,12:35:11.000,7/14/2022,31.380208,73.085924,-90.300003,2022-07-14 12:35:11,1


In [27]:
# Required unique Group **numbers** having specific cluster_members into a list
group_id = red_df["Group"].value_counts()[red_df["Group"].value_counts() >= cluster_members].index.to_list() 
group_id

[1]

In [28]:
# All records having those unique groups
red_df = red_df.loc[red_df["Group"].isin(group_id)]      

In [17]:
final_red = red_df.drop(columns=["era","Group"])

In [18]:
final_red

,Time,Date,Latitude,Longitude,frequency
28412,12:35:05.000,7/14/2022,31.379836,73.085556,-90.900002
28413,12:35:07.000,7/14/2022,31.379958,73.085677,-91.599998
28414,12:35:09.000,7/14/2022,31.380079,73.085810,-91.400002
28415,12:35:11.000,7/14/2022,31.380208,73.085924,-90.300003


In [19]:
final_df = pd.concat([green_df,final_red],ignore_index=True,axis=0)

In [20]:
final_df.query("RxLevFull (dBm) - .Server <  -90")

,Time,Date,Latitude,Longitude,frequency
29776,12:35:05.000,7/14/2022,31.379836,73.085556,-90.900002
29777,12:35:07.000,7/14/2022,31.379958,73.085677,-91.599998
29778,12:35:09.000,7/14/2022,31.380079,73.085810,-91.400002
29779,12:35:11.000,7/14/2022,31.380208,73.085924,-90.300003


In [22]:
print( " Project Done")

 Project Done


In [29]:
final_df.to_csv("filtered rxlevels.csv")

In [28]:
x = df.groupby([pd.Grouper(key='Time', freq='20S')]).frequency.count()

#x.sort_values(ascending=False)[:15]

Time
2022-07-07 00:47:40    1
2022-07-07 00:48:00    0
2022-07-07 00:48:20    0
2022-07-07 00:48:40    0
2022-07-07 00:49:00    0
                      ..
2022-07-14 10:49:00    3
2022-07-14 10:49:20    0
2022-07-14 10:49:40    0
2022-07-14 10:50:00    0
2022-07-14 10:50:20    2
Freq: 20S, Name: frequency, Length: 32049, dtype: int64

In [37]:
df.set_index('Time').resample('2s').count()

,Latitude,Longitude,frequency
Time,,,
2022-07-07 00:47:48,1,1,1
2022-07-07 00:47:50,0,0,0
2022-07-07 00:47:52,0,0,0
2022-07-07 00:47:54,0,0,0
2022-07-07 00:47:56,0,0,0
...,...,...,...
2022-07-14 10:50:22,0,0,0
2022-07-14 10:50:24,0,0,0
2022-07-14 10:50:26,0,0,0
